In [5]:

import pandas as pd
import folium
from folium.plugins import HeatMap

# 파일 경로 설정 (본인의 환경에 맞게 수정 필요)
logs_path = "/Users/woo/Downloads/archive/seoul-metro-2021.logs.csv"
stations_path = "/Users/woo/Downloads/archive/seoul-metro-station-info.csv"

logs = pd.read_csv(logs_path)
stations = pd.read_csv(stations_path)

stations.rename(columns={'station.code': 'station_code'}, inplace=True)

logs['timestamp'] = pd.to_datetime(logs['timestamp'], errors='coerce')
logs['hour'] = logs['timestamp'].dt.hour

morning = logs[logs['hour'] < 9]
evening = logs[logs['hour'] >= 17]

morning_grouped = morning.groupby('station_code')[['people_in', 'people_out']].sum()
evening_grouped = evening.groupby('station_code')[['people_in', 'people_out']].sum()

stations_info = stations[['station_code', 'geo.latitude', 'geo.longitude']].drop_duplicates()
stations_info.set_index('station_code', inplace=True)

merged_morning = morning_grouped.join(stations_info).dropna()
merged_evening = evening_grouped.join(stations_info).dropna()

def generate_heatmap(dataframe, value_column, output_file):
    map_center = [37.5665, 126.9780]  # 서울 중심
    m = folium.Map(location=map_center, zoom_start=11)
    heat_data = [
        [row['geo.latitude'], row['geo.longitude'], row[value_column]]
        for idx, row in dataframe.iterrows()
    ]
    HeatMap(heat_data).add_to(m)
    m.save(output_file)

generate_heatmap(merged_morning, 'people_in', '출근_승차_히트맵.html')
generate_heatmap(merged_evening, 'people_out', '퇴근_하차_히트맵.html')
